In [4]:
import pandas as pd

In [5]:
file_path = 'questions_data.xlsx'  # Replace with your actual file path

# staggingTables
userID=5
quizId = 22946
postTableName= "wp_jrf23a6sz5_posts"
qaTableName = "wp_jrf23a6sz5_learnpress_question_answers"
postmetaTableName="wp_jrf23a6sz5_postmeta"
quizQuestion="wp_jrf23a6sz5_learnpress_quiz_questions"

# # localTable
# userID=2
# quizId = 25469
# postTableName= "wp_posts"
# qaTableName = "wp_learnpress_question_answers"
# postmetaTableName="wp_postmeta"
# quizQuestion="wp_learnpress_quiz_questions"

# Define the base SQL template for posts table

sql_post_template = """
INSERT INTO {postTableName}
(`post_author`, `post_date`, `post_date_gmt`, `post_content`, `post_title`, 
 `post_excerpt`, `post_status`, `comment_status`, `ping_status`, `post_password`, 
 `post_name`, `to_ping`, `pinged`, `post_modified`, `post_modified_gmt`, 
 `post_content_filtered`, `post_parent`, `guid`, `menu_order`, `post_type`, 
 `post_mime_type`, `comment_count`) VALUES 
({userID}, 
 NOW(), 
 NOW(), 
 '{post_content}', 
 '{post_title}', 
 '', 
 'Publish', 
 'closed', 
 '', 
 '', 
 'AWS ML Question-{id}', 
 '', 
 '', 
 NOW(), 
 NOW(), 
 '', 
 0, 
 'http://corporateschool.local/?post_type=lp_question&p={id}', 
 0, 
 'lp_question', 
 '', 
 0);
"""

# Define the base SQL template for answers table
sql_answer_template = """
INSERT INTO {qaTableName} (`question_id`, `title`, `value`, `order`, `is_true`)  VALUES 
({question_id}, '{title}', '{value}', {order}, {is_true});
"""


quiz_question_sql = """ 
insert into `{quizQuestion}` (`quiz_id`,`question_id`,`question_order`) VALUES
({quiz_id},@lastQuestionId,{question_order});
"""


post_meta_sql = """ 
insert into {postmetaTableName} (`post_id`,`meta_key`,`meta_value`) VALUES
(@lastQuestionId, '_lp_mark',1),(@lastQuestionId, '_lp_type','{_lp_type}'); 
"""

In [6]:
df = pd.read_excel(file_path)

# Function to generate the SQL insert script for both tables
def generate_sql_script(df):
    post_answers_mapping = {}
    
    for index, row in df.iterrows():
        post_content = row['Question Text (HTML)']
        post_title = "AWS ML Question " + str(row['Question no.'])
        post_id = index + 1  # Assuming auto-increment starts at 1, adjust if needed
        
        # Generate SQL for posts table
        post_sql = sql_post_template.format(userID=userID,postTableName=postTableName,post_content=post_content.replace("'", "''"), 
                                            post_title=post_title.replace("'", "''"), 
                                            id=post_id)
        
        correct_answers = str(row['Correct answer(s)']).split(',')
        correct_answers = [answer.strip() for answer in correct_answers]  # Clean up spaces

        answer_sqls = []
        is_true_count = 0
        option_texts = []  # To store the option texts (to check for 'Yes'/'No')
        
        for answer_order in range(1, 6):  # Assuming maximum 5 options (Option 1 to Option 5)
            option_col = f'Option {answer_order}'
            
            if option_col in row and pd.notna(row[option_col]):
                answer_text = row[option_col]
                option_texts.append(answer_text.strip().lower())  # Store for 'Yes'/'No' check
                
                # Check if the current option (e.g., "B", "D") is listed as a correct answer
                is_true = "'yes'" if chr(64 + answer_order) in correct_answers else 0
                
                if is_true == "'yes'":
                    is_true_count += 1  # Count the correct answers
                
                # Generate SQL for answers table
                answer_sql = sql_answer_template.format(qaTableName=qaTableName, question_id="@lastQuestionId",
                                                        title=answer_text.replace("'", "''"),
                                                        value=answer_text.replace("'", "''"),  # Assuming title and value are the same
                                                        order=answer_order,
                                                        is_true=is_true)
                answer_sqls.append(answer_sql)
        
        # Determine the _lp_type based on conditions
        if is_true_count > 1:
            _lp_type = 'multi_choice'
        elif len(option_texts) == 2 and 'yes' in option_texts and 'no' in option_texts:
            _lp_type = 'true_or_false'
        elif is_true_count == 1:
            _lp_type = 'single_choice'
        else:
            _lp_type = 'single_choice'  # Default to single_choice if none of the conditions match
        
        post_answers_mapping[post_sql] = {'answer': answer_sqls, '_lp_type': _lp_type}

    return post_answers_mapping

# Generate SQL scripts
post_answers_mapping_script = generate_sql_script(df)


In [7]:

# Save the SQL scripts to a file
output_file_posts = 'SqlOutput/insert_posts_statements1.sql'
question_order_value = 1 
with open(output_file_posts, 'w', encoding='utf-8') as f_posts:
    for question_sql, data in post_answers_mapping_script.items():
        f_posts.write(question_sql + ' \n') # Inser questions into wp_posts
        f_posts.write("set @lastQuestionId = Last_Insert_ID();" + ' \n') #Save newly genrated question id        
        f_posts.write(quiz_question_sql.format(quizQuestion=quizQuestion,question_order=question_order_value,quiz_id=quizId)+ ' \n') # Map question with quiz
        question_order_value += 1      
        for answer_sql in data['answer']:
            f_posts.write(answer_sql + ' \n')            
            f_posts.write(' \n')         
        
        # Write the post_meta_sql with the correct _lp_type
        f_posts.write(post_meta_sql.format(postmetaTableName=postmetaTableName,_lp_type=data['_lp_type']) + ' \n' )
        # f_posts.write("commit;" + '\n')
        
        # f_posts.write('\n' + "#-------------Adding next question ----------------#" + '\n')

print(f"SQL insert scripts for posts have been written to {output_file_posts}")


SQL insert scripts for posts have been written to SqlOutput/insert_posts_statements1.sql
